In [1]:
import tensorflow as tf
import numpy as np

In [2]:
cifar = tf.keras.datasets.cifar10
(train_images, train_labels), (test_images, test_labels) = cifar.load_data()

In [3]:
def process_data(images, labels, splits):
    images = images.reshape(images.shape[0], 32, 32, 3)
    images = images.astype('float32')
    images /= 255
    labels = labels.astype(np.int32)
    labels = tf.one_hot(tf.constant(labels), 10)
    if splits == 0:
        labels = np.array(labels)
        labels = labels.reshape(labels.shape[0], 10)
        return images, labels
    else:
        split_images = np.array_split(images, splits)
        split_labels = np.array_split(labels, splits)
        reshaped_labels = []
        for label_portion in split_labels:
            reshaped_labels.append(label_portion.reshape(label_portion.shape[0], 10))
    return split_images, reshaped_labels

def model():
    input_shape = (32, 32, 3)
    inputs = tf.keras.layers.Input(shape=input_shape)
    x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = tf.keras.layers.MaxPooling2D((2, 2))(x)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(64, activation='relu')(x)
    outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
    model = tf.keras.models.Model(inputs, outputs)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

def federated_average(weight1, weight2):
    new_weights = np.array(weight1) + np.array(weight2)
    new_weights = new_weights / 2
    return new_weights.tolist()

In [4]:
split_images, split_labels = process_data(train_images, train_labels,3)

server_model = model()
client_1 = model()
client_2 = model()

server_model.fit(split_images[0], split_labels[0], epochs=1)
server_weights = server_model.get_weights()

client_1.set_weights(server_weights)
client_2.set_weights(server_weights)

client_1.fit(split_images[1], split_labels[1], epochs=1)
client_1_weights = client_1.get_weights()

client_2.fit(split_images[2], split_labels[2], epochs=1)
client_2_weights = client_2.get_weights()

global_update = federated_average(client_1_weights,client_2_weights)
server_model.set_weights(global_update)
test_images_proc, test_labels_proc = process_data(test_images, test_labels,0)
loss, accuracy = server_model.evaluate(test_images_proc, test_labels_proc)

Metal device set to: Apple M1 Pro

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB



2023-02-23 14:44:17.829214: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-23 14:44:17.829341: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
2023-02-23 14:44:18.195893: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-23 14:44:18.357016: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  6/521 [..............................] - ETA: 5s - loss: 1.4914 - accuracy: 0.4844  

2023-02-23 14:44:24.997881: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


  6/521 [..............................] - ETA: 6s - loss: 1.4177 - accuracy: 0.4688  

...

521/521 [==============================] - 6s 12ms/step - loss: 1.4053 - accuracy: 0.5048


/var/folders/wg/0kpf4bq56tn3zqg0vq6863980000gn/T/ipykernel_18410/3882461726.py:32: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  new_weights = np.array(weight1) + np.array(weight2)


 12/313 [>.............................] - ETA: 2s - loss: 1.2966 - accuracy: 0.5755

2023-02-23 14:44:38.001013: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


313/313 [==============================] - 3s 9ms/step - loss: 1.2830 - accuracy: 0.5527
